# Resistance/Support AND Candles Patterns

In [15]:
import pandas as pd
df2016=pd.read_csv("./histo_cotation_2016.csv")
df2017=pd.read_csv("./histo_cotation_2017.csv")
df2018=pd.read_csv("./histo_cotation_2018.csv")
df2019=pd.read_csv("./histo_cotation_2019.csv")
df2020=pd.read_csv("./histo_cotation_2020.csv")
df = pd.concat([df2016,df2017,df2018,df2019,df2020]).reset_index(drop=True)
#Check if NA values are in data


df.isna().sum()
df.tail()

,SEANCE,GROUPE,CODE,VALEUR,OUVERTURE,CLOTURE,PLUS_BAS,PLUS_HAUT,QUANTITE_NEGOCIEE,NB_TRANSACTION,CAPITAUX,IND_RES,C_GR_RLC,CODE_VAL,LIB_VAL,COURS_REF,COURS_VEILLE,DERNIER_COURS,NB_TRAN,I
552344,31/12/2020,52.0,760001.0,SOTEMAIL,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552345,31/12/2020,52.0,766001.0,MAGHREB INTERN PUB,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552346,31/12/2020,52.0,770001.0,OFFICEPLAST,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552347,31/12/2020,52.0,773001.0,SANIMED,1.75,1.75,1.75,1.75,2300,6.0,4025.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552348,31/12/2020,99.0,503001.0,STIP,0.92,0.92,0.92,0.92,400,1.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Support and Resistance FUNCTIONS

In [16]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.PLUS_BAS[i]>df1.PLUS_BAS[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.PLUS_BAS[i]<df1.PLUS_BAS[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.PLUS_HAUT[i]<df1.PLUS_HAUT[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.PLUS_HAUT[i]>df1.PLUS_HAUT[i-1]):
            return 0
    return 1

In [17]:
length = len(df)
high = list(df['PLUS_HAUT'])
low = list(df['PLUS_BAS'])
close = list(df['CLOTURE'])
open = list(df['OUVERTURE'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.PLUS_HAUT[l]-min(levels, key=lambda x:abs(x-df.PLUS_HAUT[l])))<=lim
    c2 = abs(max(df.OUVERTURE[l],df.CLOTURE[l])-min(levels, key=lambda x:abs(x-df.PLUS_HAUT[l])))<=lim
    c3 = min(df.OUVERTURE[l],df.CLOTURE[l])<min(levels, key=lambda x:abs(x-df.PLUS_HAUT[l]))
    c4 = df.PLUS_BAS[l]<min(levels, key=lambda x:abs(x-df.PLUS_HAUT[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.PLUS_BAS[l]-min(levels, key=lambda x:abs(x-df.PLUS_BAS[l])))<=lim
    c2 = abs(min(df.OUVERTURE[l],df.CLOTURE[l])-min(levels, key=lambda x:abs(x-df.PLUS_BAS[l])))<=lim
    c3 = max(df.OUVERTURE[l],df.CLOTURE[l])>min(levels, key=lambda x:abs(x-df.PLUS_BAS[l]))
    c4 = df.PLUS_HAUT[l]>min(levels, key=lambda x:abs(x-df.PLUS_BAS[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [18]:
n1=2
n2=2
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.PLUS_BAS[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.PLUS_HAUT[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



TypeError: '<' not supported between instances of 'str' and 'float'

In [ ]:
df['signal']=signal

In [ ]:
df[df['signal']==2].count()

In [ ]:
def SIGNAL():
    return df.signal

In [ ]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [ ]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.CLOTURE[-1] - 750e-4
            tp1 = self.data.CLOTURE[-1] + 600e-4
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.CLOTURE[-1] + 750e-4
            tp1 = self.data.CLOTURE[-1] - 600e-4
            self.sell(sl=sl1, tp=tp1)

In [ ]:
from backtesting import Backtest

bt = Backtest(df, MyCandlesStrat, cash=500, commission=.08)
stat = bt.run()
stat



In [ ]:
bt.plot()